In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px
import numpy as np
import seaborn as sns
import plotly.express as px
import requests

In [ ]:
df = pd.read_parquet('dataset_map.parquet')

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100) 

df.head()

In [ ]:
df.shape

#### Dernière vérification des colonnes des Dates

In [ ]:
# df["Date de la dernière mise à jour de l'établissement"] = pd.to_datetime(
#     df["Date de la dernière mise à jour de l'établissement"]
# )
# date_min = df["Date de la dernière mise à jour de l'établissement"].min()
# date_max = df["Date de la dernière mise à jour de l'établissement"].max()

# print(f"Fourchette des dates de mise à jour : {date_min.date()} → {date_max.date()}")

# # Colonne à droper

In [ ]:
# df["Date du début de la période de l'établissement"] = pd.to_datetime(
#     df["Date du début de la période de l'établissement"]
# )
# date_min = df["Date du début de la période de l'établissement"].min()
# date_max = df["Date du début de la période de l'établissement"].max()

# print(f"Fourchette des dates du début de la période : {date_min.date()} → {date_max.date()}")

# # Colonne à droper

In [ ]:
# df["Date du dernier traitement de l'unité légale"] = pd.to_datetime(
#     df["Date du dernier traitement de l'unité légale"]
# )
# date_min = df["Date du dernier traitement de l'unité légale"].min()
# date_max = df["Date du dernier traitement de l'unité légale"].max()

# print(f"Fourchette des dates du traitement de l'unité : {date_min.date()} → {date_max.date()}")

# # Colonne à droper

In [ ]:
df["Caractère employeur de l'établissement"].value_counts()

---

#### Drop des colonnes inutiles

In [ ]:
cols_to_drop = ["Date de la dernière mise à jour de l'établissement",
                "Date du début de la période de l'établissement",
                "Commune de l'établissement",
                "Date du dernier traitement de l'unité légale",
                "Code EPCI de l'établissement",
                "Libellé de l'EPCI de l'établissement",
                "Département de l'établissement",
                "Région de l'établissement",
                "Nature juridique de l'unité légale",
                "Adresse de l'établissement",
                "Groupe de l'établissement",
                "Activité principale de l'établissement.1",
                
]

df.drop(columns=cols_to_drop, inplace=True)
df.head()

In [ ]:
df.shape

---

#### Traitement des années d'existence

In [ ]:
# visu de la distribution
fig = px.box(
    df,
    y="age_estime",
    title="Boxplot de l'age des sociétés"
)

fig.show()

#### Conservation des outliers, mais prévoir une catégorisation pour le modèle, dans le notebook approprié.

---

#### Traitement de l'activité

In [ ]:
ape_counts=df["Activité principale de l'unité légale"].value_counts()
ape_percent = ape_counts / ape_counts.sum() * 100

# Voir les 30 premiers en pourcentage
ape_percent.head(30)

In [ ]:
top_30 = ape_counts.sort_values(ascending=False).head(30)

plt.figure(figsize=(10,8))
plt.barh(top_30.index[::-1], top_30.values[::-1], color='skyblue')
plt.xlabel("Nombre d'établissements")
plt.ylabel("Code APE")
plt.title("Top 30 des codes APE les plus fréquents des sociétés fermées")
plt.tight_layout()
plt.show()

---

#### Import des codes APE officiels pour analyse des secteurs

In [ ]:
ape=pd.read_csv('codes_ape.csv')
ape.head()

In [ ]:
df.columns.to_list()

In [ ]:
# --- Étape 1 : créer une colonne "ape_section" avec les 2 premiers chiffres ---

df["Activité principale de l'unité légale"] = df["Activité principale de l'unité légale"].astype(str)

# On prend les 2 premiers chiffres du code APE
df["ape_section"] = df["Activité principale de l'unité légale"].str[:2]

# --- Étape 2 : préparer le mapping avec le fichier ape ---

ape["code_ape"] = ape["code_ape"].astype(str)

# On ne garde que les sections de 2 caractères pour le mapping
ape_sections = ape[ape["code_ape"].str.len() == 2][["code_ape", "libelle_ape"]]

# --- Étape 3 : fusionner pour ajouter le libellé de la section ---
df = df.merge(ape_sections, how="left", left_on="ape_section", right_on="code_ape")

# On renomme la colonne
df = df.rename(columns={"libelle_ape": "libelle_section_ape"})

# --- Vérification ---
df[["Activité principale de l'unité légale", "ape_section", "libelle_section_ape"]].head(10)

In [ ]:
df.shape

In [ ]:
df.columns.to_list()

In [ ]:
df.head()

In [ ]:
# --- Étape 1 : compter le nombre d'entreprises par section APE ---
section_counts = df.groupby("ape_section") \
                   .size() \
                   .reset_index(name="nombre_entreprises")

# --- Étape 2 : récupérer le libellé correspondant pour chaque section ---

libelles = df.groupby("ape_section")["libelle_section_ape"].first().reset_index()
section_counts = section_counts.merge(libelles, on="ape_section")

# --- Étape 3 : trier par ordre décroissant et prendre les 20 premières ---
section_counts = section_counts.sort_values("nombre_entreprises", ascending=False).head(20)

# --- Étape 4 bis : visualiser avec un barplot interactif avec les plus grandes valeurs en haut ---
fig = px.bar(
    section_counts,
    y="libelle_section_ape",
    x="nombre_entreprises",
    title="Top 20 des sections APE par nombre d'entreprises fermées",
    labels={"libelle_section_ape": "Secteur d'activité", "nombre_entreprises": "Nombre d'entreprises"},
    text="nombre_entreprises",
    height=600,
    width=1600,
    category_orders={"libelle_section_ape": section_counts.sort_values("nombre_entreprises", ascending=False)["libelle_section_ape"].tolist()}
)


fig.update_traces(textposition="outside")

# Ajustement des marges pour les labels longs
fig.update_layout(xaxis_tickangle=-45, margin=dict(l=200))

fig.show()

In [ ]:
# --- Étape 1 : filtrer uniquement les 30 sections les plus nombreuses ---
top_sections = section_counts['ape_section'].tolist()
df_top = df[df['ape_section'].isin(top_sections)]

# --- Étape 2 : calculer l'âge moyen par section ---
age_moyenne = df_top.groupby(['ape_section', 'libelle_section_ape'])['age_estime'].mean().reset_index()
age_moyenne = age_moyenne.rename(columns={'age_estime': 'age_moyen'})

# --- Étape 3 : fusionner avec le nombre d'entreprises pour le graphique ---
age_moyenne = age_moyenne.merge(section_counts[['ape_section', 'nombre_entreprises']], on='ape_section')

# --- Étape 4 : barplot interactif avec palette rouge → vert → bleu ---
fig = px.bar(
    age_moyenne,
    y='libelle_section_ape',
    x='nombre_entreprises',
    color='age_moyen',                      
    text='age_moyen',
    title="Top 20 sections APE sociétés fermées : nombre d'entreprises et âge moyen",
    labels={
        'libelle_section_ape': "Secteur d'activité",
        'nombre_entreprises': "Nombre d'entreprises",
        'age_moyen': "Âge moyen (années)"
    },
    height=600,
    width=1500,
    category_orders={"libelle_section_ape": age_moyenne.sort_values("nombre_entreprises", ascending=False)["libelle_section_ape"].tolist()},
    color_continuous_scale=['red','green','blue'] 
)

# Rendre le texte lisible
fig.update_traces(texttemplate='%{text:.1f}', textposition='outside')

# Ajuster les marges pour les labels longs
fig.update_layout(xaxis_tickangle=-45, margin=dict(l=200))

fig.show()

---

#### Exploration durée moyenne selon les 20 secteurs les plus représentés

In [ ]:
# --- Étape 1 : calculer la moyenne du nombre de périodes par section ---
moyennes_age = df.groupby("ape_section")["age_estime"] \
                      .mean() \
                      .round(2) \
                      .reset_index(name="moyenne_age")

# --- Étape 2 : ne garder que les 20 sections les plus fréquentes ---
top20_sections = section_counts["ape_section"]
moyennes_top20 = moyennes_age[moyennes_age["ape_section"].isin(top20_sections)]

# --- Étape 3 : ajouter le libellé pour plus de lisibilité ---
moyennes_top20 = moyennes_top20.merge(
    section_counts[["ape_section", "libelle_section_ape"]],
    on="ape_section"
)

# --- Étape 4 : afficher le résultat trié par moyenne décroissante ---
moyennes_top20 = moyennes_top20.sort_values("moyenne_age", ascending=False)
moyennes_top20

In [ ]:
# --- On trie les données par moyenne décroissante ---
moyennes_top20_sorted = moyennes_top20.sort_values("moyenne_age", ascending=True)

# --- Graphique ---
fig = px.bar(
    moyennes_top20_sorted,
    y="libelle_section_ape",
    x="moyenne_age",
    title="Moyenne du nombre de périodes par secteur (Top 20)",
    labels={
        "libelle_section_ape": "Secteur d'activité",
        "moyenne_age": "Moyenne du nombre de périodes"
    },
    text="moyenne_age",
    height=600,
    width=1700,
    category_orders={"libelle_section_ape": moyennes_top20_sorted["libelle_section_ape"].tolist()}
)

# Texte à l'extérieur et ajustement des marges
fig.update_traces(textposition="outside")
fig.update_layout(
    xaxis_tickangle=-45,
    margin=dict(l=200),
    xaxis=dict(range=[6, 11.5])
)

fig.show()


---

#### Post-traitement du dataset

In [ ]:
df.head()

In [ ]:
df.shape

In [ ]:
df=df.drop(columns=['SIRET', 
                    "Caractère employeur de l'établissement", 
                    "Nombre de périodes de l'unité légale", 
                    "Section de l'établissement",
                    "Sous-section de l'établissement", 
                    "Division de l'établissement", 
                    "ape_section"])

df.head()

---

#### Rajout d'une colonne 'fermeture' avec valeur à 1 pour indiquer la fermeture

In [ ]:
df['fermeture']=1
df.head()

---

In [ ]:
df.shape

---

##### Dernière vérification des Nan

In [ ]:
df.isna().sum().sort_values(ascending=False)


In [ ]:
df = df.dropna()

In [ ]:
df.isna().sum().sort_values(ascending=False)

##### Export du dataset en csv

In [ ]:
df.dtypes

In [ ]:
# 1. On convertit en string
df['SIREN'] = df['SIREN'].astype(str)

# 2. On compte combien n'ont pas la bonne taille (9 caractères)
siren_counts = df['SIREN'].str.len().value_counts()
print("Répartition des longueurs de SIREN :")
print(siren_counts)

In [ ]:
# On force le format 9 caractères en ajoutant les zéros manquants à gauche
df['SIREN'] = df['SIREN'].astype(str).str.zfill(9)

# Petit contrôle rapide pour confirmer que tout est à 9
print(df['SIREN'].str.len().value_counts())

In [ ]:
df.shape

---

#### Export en parquet

In [ ]:
# df.to_parquet("dataset_closed_final.parquet", index=False)